This script go with ensembles of trees. rf, randomtreeembedding, gbdt

In [58]:
import numpy as np
np.random.seed(123)

In [59]:
import matplotlib.pyplot as plt

In [60]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomTreesEmbedding, RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import make_pipeline


In [61]:
n_estimator = 10
X, y = make_classification(n_samples = 80000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5)

In [62]:
X_train_s, X_train_lr, y_train_s, y_train_lr = train_test_split(X_train, y_train, test_size = 0.5)

### Unsupervised transformation based on totally random trees

In [63]:
rt = RandomTreesEmbedding(max_depth = 2, n_estimators = n_estimator, random_state = 123)
rt.fit(X_train, y_train)

RandomTreesEmbedding(max_depth=2, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, random_state=123, sparse_output=True,
           verbose=0, warm_start=False)

In [64]:
rt_lr = LogisticRegression()

In [65]:
pipeline = make_pipeline(rt, rt_lr)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('randomtreesembedding', RandomTreesEmbedding(max_depth=2, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, random_state=123, sparse_output=True,
           verbose...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [68]:
y_pred_rt_lr = pipeline.predict_proba(X_test)[:,1]

In [69]:
roc_auc_score(y_test,y_pred_rt_lr)

0.79247074135520201

### Supervised transformation based on random forests

In [70]:
rf = RandomForestClassifier(max_depth= 2, n_estimators= n_estimator, random_state=123)
rf_enc = OneHotEncoder()
rf_lr = LogisticRegression()
rf = rf.fit(X_train, y_train)
rf2 = rf.fit(X_train_s, y_train_s)
rf_enc.fit(rf2.apply(X_train_s))
rf_lr.fit(rf_enc.transform(rf2.apply(X_train_lr)), y_train_lr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [71]:
y_red_rf = rf.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_red_rf)

0.96575539779748887

In [74]:
y_red_rf_lr = rf_lr.predict_proba(rf_enc.transform(rf.apply(X_test)))[:,1]
roc_auc_score(y_test, y_red_rf_lr)

0.98001039398367651

### Supervised transformation based on GBDT

In [46]:
gbt = GradientBoostingClassifier(max_depth=2, n_estimators= n_estimator)
gbt_enc = OneHotEncoder()
gbt_lr = LogisticRegression()
gbt.fit(X_train, y_train)
gbt_enc.fit(gbt.apply(X_train)[:,:,0])
gbt_lr.fit(gbt_enc.transform(gbt.apply(X_train_lr)[:,:,0]), y_train_lr)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [50]:
y_pred_gbt = gbt.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_gbt)

0.97698693668767489

In [51]:
y_pred_gbt_lr = gbt_lr.predict_proba(gbt_enc.transform(gbt.apply(X_test)[:,:,0]))[:,1]
roc_auc_score(y_test, y_pred_gbt_lr)

0.97832809103301499

### Unsupervised transformation based on totally random trees